In [ ]:
# -*- coding: utf-8 -*-
# ======================================================================
#   Cooper, Gulen & Schill (2008) — Table III (1963–2024 전체 버전)
#   Panel C / D, Fama–MacBeth + Table 이미지/CSV 출력
# ======================================================================

!pip install wrds pandas numpy statsmodels tqdm --quiet

import wrds
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from google.colab import files

pd.set_option("display.max_columns", 200)

# ======================================================================
# WRDS CONNECT
# ======================================================================
db = wrds.Connection()

# ======================================================================
# 샘플 기간 설정
# ======================================================================
START_FYEAR = 1962
END_FYEAR   = 2023     # portyear = 2024까지 생성됨
START_CRSP_DATE = '1962-01-01'
END_CRSP_DATE   = '2024-12-31'
PORTYEAR_START  = 1963
PORTYEAR_END    = 2024

# ======================================================================
# 1. LOAD COMPUSTAT (1962–2023)
# ======================================================================
sql_comp = f"""
    SELECT gvkey, datadate, fyear, at, sale, seq, ceq, txditc,
           pstkrv, pstkl, pstk, ni, dlc, dltt, rect, act, che,
           ppent, ap, lct, lt, dp, cusip
    FROM comp.funda
    WHERE indfmt='INDL'
      AND datafmt='STD'
      AND consol='C'
      AND fyear BETWEEN {START_FYEAR} AND {END_FYEAR}
"""
comp = db.raw_sql(sql_comp, date_cols=["datadate"])
comp = comp.sort_values(["gvkey", "fyear"])

# txditc missing 처리
if "txditc" not in comp.columns:
    comp["txditc"] = 0

# ========== Book Equity ==========
comp["pstk_adj"] = comp["pstkrv"].fillna(comp["pstkl"]).fillna(comp["pstk"])
comp["be"] = comp["seq"]

mask = comp["be"].isna()
comp.loc[mask, "be"] = (
    comp.loc[mask, "ceq"]
    + comp.loc[mask, "txditc"].fillna(0)
    - comp.loc[mask, "pstk_adj"].fillna(0)
)
comp = comp[comp["be"] > 0]

# BM lag
comp["be_lag1"] = comp.groupby("gvkey")["be"].shift(1)

# ======================================================================
# 1.1 LAG VARIABLES
# ======================================================================
lag_vars = ["at", "sale", "be", "ppent", "act", "lct"]
for v in lag_vars:
    for L in [1, 2, 5]:
        comp[f"{v}_lag{L}"] = comp.groupby("gvkey")[v].shift(L)

# ======================================================================
# 1.2 VARIABLE FORMULAS
# ======================================================================
comp["assetg"]   = (comp["at"]   - comp["at_lag1"])   / comp["at_lag1"]
comp["l2assetg"] = (comp["at_lag1"] - comp["at_lag2"]) / comp["at_lag2"]

comp["sysalesg"] = (comp["sale"] - comp["sale_lag5"]) / comp["sale_lag5"]
comp["syassetg"] = (comp["at"]   - comp["at_lag5"])   / comp["at_lag5"]

comp["ci"] = (comp["ppent"] - comp["ppent_lag1"]) / comp["ppent_lag1"]

comp["noa"]   = comp["at"] - comp["che"] - comp["dlc"] - comp["dltt"] - comp["ni"]
comp["noa_a"] = comp["noa"] / comp["at"]

comp["accruals"] = (
    (comp["act"] - comp["act_lag1"])
    - (comp["lct"] - comp["lct_lag1"])
    - comp["dp"].fillna(0)
) / comp["at"]

comp["portyear"] = comp["fyear"] + 1
comp["cusip8"]   = comp["cusip"].str[:8]

# ======================================================================
# 2. CUSIP LINK → PERMNO (msenames만 사용)
# ======================================================================
sql_link = """
    SELECT permno, ncusip, namedt, nameendt
    FROM crsp.msenames
    WHERE ncusip IS NOT NULL
"""
msenames = db.raw_sql(sql_link, date_cols=["namedt", "nameendt"])
msenames["cusip8"] = msenames["ncusip"].str[:8]

comp_linked = comp.merge(
    msenames[["permno", "cusip8", "namedt", "nameendt"]],
    on="cusip8", how="inner"
)

comp_linked = comp_linked[
    (comp_linked["datadate"] >= comp_linked["namedt"]) &
    (comp_linked["datadate"] <= comp_linked["nameendt"])
]

# ======================================================================
# 3. LOAD CRSP (1962–2024)
# ======================================================================
sql_msf = f"""
    SELECT a.permno, a.date, a.ret, a.prc, a.shrout,
           b.shrcd, b.exchcd
    FROM crsp.msf a
    LEFT JOIN crsp.msenames b
         ON a.permno = b.permno
        AND a.date BETWEEN b.namedt AND b.nameendt
    WHERE a.date BETWEEN '{START_CRSP_DATE}' AND '{END_CRSP_DATE}'
"""
crsp = db.raw_sql(sql_msf, date_cols=["date"])
crsp["ret"] = pd.to_numeric(crsp["ret"], errors="coerce")

crsp = crsp[(crsp["shrcd"].isin([10, 11])) & (crsp["exchcd"].isin([1, 2, 3]))]

# delisting return
sql_dl = "SELECT permno, dlret, dlstdt FROM crsp.msedelist"
dl = db.raw_sql(sql_dl, date_cols=["dlstdt"])
dl["dlret"] = pd.to_numeric(dl["dlret"], errors="coerce").fillna(0)

crsp = crsp.merge(
    dl, left_on=["permno", "date"], right_on=["permno", "dlstdt"], how="left"
)

crsp["dlret"] = crsp["dlret"].fillna(0)
crsp["ret_adj"] = (1 + crsp["ret"]) * (1 + crsp["dlret"]) - 1

crsp["me"]    = crsp["prc"].abs() * crsp["shrout"] / 1000
crsp["year"]  = crsp["date"].dt.year
crsp["month"] = crsp["date"].dt.month
crsp["portyear"] = np.where(crsp["month"] >= 7, crsp["year"], crsp["year"] - 1)

# ======================================================================
# 4. JUNE ME / BHRET6 / BHRET36 / Annual Return
# ======================================================================
crsp_june = crsp[crsp["month"] == 6][["permno", "year", "me", "exchcd"]]
crsp_june = crsp_june.rename(columns={"year": "portyear", "me": "me_june"})

bh6 = (
    crsp[crsp["month"] <= 6]
    .groupby(["permno", "year"])["ret_adj"]
    .apply(lambda x: np.prod(1 + x) - 1)
    .reset_index()
    .rename(columns={"year": "portyear", "ret_adj": "bhret6"})
)

crsp = crsp.sort_values(["permno", "date"])
crsp["cum"] = (1 + crsp["ret_adj"]).groupby(crsp["permno"]).cumprod()

bh36 = crsp[crsp["month"] == 6].copy()
bh36["lag"] = bh36.groupby("permno")["cum"].shift(36)
bh36["bhret36"] = bh36["cum"] / bh36["lag"] - 1
bh36 = bh36[["permno", "year", "bhret36"]].rename(columns={"year": "portyear"})

ann = (
    crsp.groupby(["permno", "portyear"])["ret_adj"]
    .apply(lambda x: np.prod(1 + x) - 1)
    .reset_index()
    .rename(columns={"ret_adj": "ret_annual"})
)

# ======================================================================
# 5. MERGE ALL DATA
# ======================================================================
df = comp_linked.merge(crsp_june, on=["permno", "portyear"], how="inner")
df = df.merge(bh6,  on=["permno", "portyear"], how="left")
df = df.merge(bh36, on=["permno", "portyear"], how="left")
df = df.merge(ann,  on=["permno", "portyear"], how="inner")

df = df[(df["portyear"] >= PORTYEAR_START) & (df["portyear"] <= PORTYEAR_END)]

df["bm"] = df["be_lag1"] / df["me_june"]
df["bm"] = df["bm"].replace([np.inf, -np.inf], np.nan)

print("포트폴리오 연도 범위:", df["portyear"].min(), "→", df["portyear"].max())

# ======================================================================
# 6. NYSE BREAKPOINTS
# ======================================================================
nyse_bp = (
    df[df["exchcd"] == 1]
    .groupby("portyear")["me_june"]
    .quantile([0.3, 0.7])
    .unstack()
    .rename(columns={0.3: "bp30", 0.7: "bp70"})
)

df = df.merge(nyse_bp, on="portyear", how="left")

def size_group(row):
    if pd.isna(row["me_june"]):
        return None
    if row["me_june"] <= row["bp30"]:
        return "Small"
    if row["me_june"] <= row["bp70"]:
        return "Medium"
    return "Large"

df["size_group"] = df.apply(size_group, axis=1)

dfC = df[df["size_group"] == "Medium"].copy()
dfD = df[df["size_group"] == "Large"].copy()

print("Medium obs:", len(dfC), " / Large obs:", len(dfD))

# ======================================================================
# 7. WINSORIZATION
# ======================================================================
winsor_cols = [
    "assetg","l2assetg","bm","me_june","bhret6","bhret36",
    "sysalesg","syassetg","ci","noa_a","accruals"
]

def winsorize(df, cols):
    for c in cols:
        q_low  = df[c].quantile(0.01)
        q_high = df[c].quantile(0.99)
        df[c] = df[c].clip(lower=q_low, upper=q_high)
    return df

dfC = winsorize(dfC, winsor_cols)
dfD = winsorize(dfD, winsor_cols)

# ======================================================================
# 8. FAMA–MACBETH
# ======================================================================
def newey_west(betas, lag=1):
    betas = np.asarray(betas)
    X = betas - betas.mean(0)
    T = betas.shape[0]
    S = (X.T @ X) / T
    for L in range(1, lag + 1):
        w = 1 - L/(lag + 1)
        Gamma = (X[L:].T @ X[:-L]) / T
        S += w * (Gamma + Gamma.T)
    return S

def fama_macbeth(df, Xvars):
    betas = []
    for yr, g in df.groupby("portyear"):
        cols = Xvars + ["ret_annual"]
        g2 = g[cols].astype(float).dropna()
        if g2.shape[0] < len(Xvars) + 2:
            continue
        X = sm.add_constant(g2[Xvars].values, has_constant='add')
        y = g2["ret_annual"].values
        m = sm.OLS(y, X).fit()
        params = np.asarray(m.params)
        if np.isnan(params).any():
            continue
        betas.append(params)

    if len(betas) == 0:
        K = len(Xvars) + 1
        return np.full(K, np.nan), np.full(K, np.nan)

    betas = np.vstack(betas)
    beta_mean = betas.mean(0)
    cov = newey_west(betas)
    tstat = beta_mean / np.sqrt(np.diag(cov))
    return beta_mean, tstat

# ======================================================================
# 9. MODEL VARIABLES
# ======================================================================
MODEL_VARS = {
    "M1": ["assetg","bm","me_june","bhret6","bhret36"],
    "M2": ["assetg","l2assetg","bm","me_june","bhret6","bhret36"],
    "M3": ["assetg","bm","me_june","bhret6","bhret36","sysalesg"],
    "M4": ["assetg","bm","me_june","bhret6","bhret36","ci"],
    "M5": ["assetg","bm","me_june","bhret6","bhret36","noa_a"],
    "M6": ["assetg","bm","me_june","bhret6","bhret36","accruals"],
    "M7": ["assetg","bm","me_june","bhret6","bhret36","syassetg"]
}

resultsC = {m: fama_macbeth(dfC, X) for m, X in MODEL_VARS.items()}
resultsD = {m: fama_macbeth(dfD, X) for m, X in MODEL_VARS.items()}

# ======================================================================
# 10. TABLE 출력
# ======================================================================
TABLE_COLS = [
    "Constant","ASSETG","L2ASSETG","BM","MV",
    "BHRET6","BHRET36","5YSALESG","CI","NOA/A",
    "ACCRUALS","5YASSETG"
]

VAR_MAP = {
    "const": "Constant",
    "assetg": "ASSETG",
    "l2assetg": "L2ASSETG",
    "bm": "BM",
    "me_june": "MV",
    "bhret6": "BHRET6",
    "bhret36": "BHRET36",
    "sysalesg": "5YSALESG",
    "ci": "CI",
    "noa_a": "NOA/A",
    "accruals": "ACCRUALS",
    "syassetg": "5YASSETG",
}

def build_two_rows(beta, tval, Xvars):
    beta_row = {c: "." for c in TABLE_COLS}
    t_row    = {c: "." for c in TABLE_COLS}

    fullvars = ["const"] + Xvars
    for name, b, t in zip(fullvars, beta, tval):
        col = VAR_MAP[name]
        if np.isnan(b):
            beta_row[col] = "nan"
            t_row[col]    = "(nan)"
        else:
            beta_row[col] = f"{b:.6f}"
            t_row[col]    = f"({t:.2f})"
    return beta_row, t_row

# ======================================================================
# 11. IMAGE & CSV 저장
# ======================================================================
def save_table_image(panel_name, results, filename):
    header = ["Model", "Type"] + TABLE_COLS
    table_data = []

    for i, (m, (beta, tval)) in enumerate(results.items(), start=1):
        row_beta, row_t = build_two_rows(beta, tval, MODEL_VARS[m])
        table_data.append([str(i), "Beta"]   + [row_beta[c] for c in TABLE_COLS])
        table_data.append(["",      "t-stat"]+ [row_t[c]    for c in TABLE_COLS])
        table_data.append([""] * len(header))

    fig, ax = plt.subplots(figsize=(22, len(table_data) * 0.42))
    ax.axis("off")

    the_table = plt.table(
        cellText=[header] + table_data,
        cellLoc="left",
        loc="center"
    )

    the_table.auto_set_font_size(False)
    the_table.set_fontsize(10)
    the_table.scale(1, 1.25)

    for j in range(len(header)):
        cell = the_table[0, j]
        cell.set_facecolor("#F0F0F0")
        cell.set_text_props(weight="bold")

    for (r, c), cell in the_table.get_celld().items():
        cell.set_linewidth(0)

    plt.title(panel_name, fontsize=18, fontweight='bold', pad=15)
    plt.savefig(filename, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"Saved → {filename}")

def save_table_csv(results, filename_csv):
    header = ["Model", "Type"] + TABLE_COLS
    rows = []
    for i, (m, (beta, tval)) in enumerate(results.items(), start=1):
        row_beta, row_t = build_two_rows(beta, tval, MODEL_VARS[m])
        rows.append([str(i), "Beta"]   + [row_beta[c] for c in TABLE_COLS])
        rows.append(["",      "t-stat"]+ [row_t[c]    for c in TABLE_COLS])
    df_out = pd.DataFrame(rows, columns=header)
    df_out.to_csv(filename_csv, index=False, encoding="utf-8-sig")
    print(f"Saved CSV → {filename_csv}")

# 전체 기간 Panel C / D 저장
save_table_image("Panel C. Medium Size Firms (1963–2024)", resultsC, "PanelC_Table3_1963_2024.png")
save_table_image("Panel D. Large Size Firms (1963–2024)", resultsD, "PanelD_Table3_1963_2024.png")

save_table_csv(resultsC, "PanelC_Table3_1963_2024.csv")
save_table_csv(resultsD, "PanelD_Table3_1963_2024.csv")

files.download("PanelC_Table3_1963_2024.png")
files.download("PanelD_Table3_1963_2024.png")
files.download("PanelC_Table3_1963_2024.csv")
files.download("PanelD_Table3_1963_2024.csv")
